In [ ]:
# PLEASE READ THE "Read me" DOCUMENT ATTACHED WITH THIS FILE, IT PROVIDES IN-DEPTH EXPLANTION OF THE TERMINOLOGY WHICH WILL
# VERY USEFUL FOR UNDERSTANDING THE COMMENTS

# Welcome to Arsh Suri's Quantum Estimator which predicts the values of pi and any n-th root using Quantum Phase Estimation (QPE)
# algorithms and Quantum Circuits.

# To use this program make sure to check the Read Me document attached with this program.
# There will be an in-depth explanation on how to run it if you so wanted. Make sure to replace your filler API key with
# your own if you are running it. Quantum Computers are highly susceptible to noise so the results may vary drastically if
# running it on a separate computer.

# Importing required libraries and giving easy access for usage. The libraries are qiskit, numpy, matplotlib, iqx
# These are used for general quantum computing, math, and plotting.


from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from iqx import *
from qiskit import *
import numpy as np
import matplotlib.pyplot as plt
from qiskit.tools.monitor import job_monitor

# This code is for initializing the actual simulation on IBM Q 14 Melbourne (A IBM Quantum processor) over the cloud.
# To run it make sure you place your api key in the "api" section. Make sure to keep the quotations. For getting your api
# key, go to the Read Me Doc which should be submitted.
IBMQ.save_account(
    "6a68158e0f007b8d61f8ca05e05e9fd949e591dd7607be1b426843162c23c817728e070fe37d054db4e5b99887bbec4b90ccfbd806f2091a4e1bbcacce85b878",
    overwrite=True)
IBMQ.load_account()
my_provider = IBMQ.get_provider()
simulator_cloud = my_provider.get_backend('ibmq_qasm_simulator')
device = my_provider.get_backend('ibmq_16_melbourne')
simulator = Aer.get_backend('qasm_simulator')

# Initializes the constant pi, we mostly just need this for initializing quantum circuits. This does not actually change the
# the value for the quantum phase estimation of pi. However, this value is used when quantum phase estimating n-th roots.
pi = np.pi
maxqubits = 15

#Sets global variables for userinput (which stores userinput) and boolPi(if the input is pi or not) so they can be edited by functions
global userinput
global boolPi

# Defining function to take user input and return the parameters for the radical or pi
def getUserinput():
    global userinput
    global boolPi
    print(
        "Welcome to Arsh Suri's Quantum Estimator where you can use Quantum Phase Estimation to estimate irrational numbers. : ")
    print("This ranges from pi or any n-th root number!")

    # Asks user for whether they want to estimate root or pi
    while True:
        # Cleans out the input for asking if they want to estimate roots or pi, returns pi instantly because there are no further parameters for pi
        category = input(
            "What would you like to estimate? What would you like to estimate first? Pi or a n-th root? [Type: 'pi' for estimation of pi or Type: 'root' for estimation of a n-th root : ")
        if category == "pi":
            # returns as pi, pi incase they were errors for list indexing
            print("\n")
            userinput = ['pi', 'pi']
            boolPi = True
            return userinput
        if category == "root":
            # This entire section is for cleaning the input and asking the input for what index and what radicand they want to estimate
            while True:
                index = input(
                    "What radical index would you like to estimate? Just type in the number, keep these in decimal form. [Remember that a 'square' root has a index of 2 and a 'cube' root has a index of 3] : ")
                try:
                    index = float(index)
                    while True:
                        radicand = input("Which number did you want to " + str(index) + "-th root? Keep this positive and in decimal form : ")
                        try:
                            radicand = float(radicand)
                            # Making sure it was positive as you can't have an irrational negative root
                            if radicand >= 0:
                                #A debugging tool to see if the numbers are properly stored and given
                                #print(index, radicand)
                                # returns list of the index and radicand
                                print("\n")
                                userinput = [index, radicand]
                                boolPi = False
                                return userinput
                        except:
                            print("Make sure to type a number")
                            pass
                except:
                    print("Make sure to type a number in decimal form")
                    pass
        # A else statement for cleaning the first input
        else:
            print("Make sure to type 'root' or 'pi'")
            pass


# Initialize the state of the entire system before the estimation depending on the current circuit and number of qubits
def initQpe(tempCirc, numQubits):
    # Applies the Hadamard gate on all qubits
    tempCirc.h(range(numQubits))
    tempCirc.x(numQubits)

    # Reverses the elements of the list of the range of numQubits which the for loop will iterate through
    for x in reversed(range(numQubits)):
        # This section of the code ensures that the last qubit is set to a state of superpositon of |1>
        for n in range(2 ** (numQubits - 1 - x)):
            tempCirc.cu1(1, numQubits - 1 - x, numQubits)


# Runs a task on the quantum cloud processor depending on the current state of the quantum circuit and
# which backend (aka just which device we are using to run these jobs)
def run(tempCirc, backend_, shots_=1000, optimization_level_=0):
    # job is set to the return of execute which executes a circuit depending on the number of "shots". 1 "shots" = 1 repition
    job = execute(tempCirc, backend=backend_, shots=shots_, optimization_level=optimization_level_)

    # monitors the progress of the job
    job_monitor(job)

    # gets the counts for the values of the quantum circuit, this usually must be done after the system collaspes when measured meaning there are definite values
    return job.result().get_counts(tempCirc)


# Defines a function which applies a reverse Quantum Fourier Transform depending on the current state of the circuit (tempCirc)
# and number of allocated qubits (numQubits). A reverse Fourier Transform basically gives you a function based off all of its parts.
# In terms of a wave, it gives the you the wavefunction based on all the wave heights it is reading.
def reverse_qft(tempCirc, numQubits):
    # For each qubit, applies the SWAP gate or essentially flips the variables in each state of the system. Ex: |x, y> to |y, x>. We do this for each pair of qubits.
    for x in range(int(numQubits / 2)):
        tempCirc.swap(x, numQubits - x - 1)

    # Now using these swapped qubits, we apply the Cu1 state this changes the value of the of the qubit by diagonally flipped qubit.
    # This causes the phase of the qubits to change
    for n in range(0, numQubits):
        for w in range(n):
            tempCirc.cu1(-np.pi / float(2 ** (n - w)), w, n)
        tempCirc.h(n)


# The actual estimation of the phase of the number we are trying to estimate
def phaseEstimate(numQubits, input):
    # Qunatum Phase Estimation is the combination of these events in order
    # creating a circuiting with the number of Qubits and classical bits.
    circ = QuantumCircuit(numQubits + 1, numQubits)

    # Intilizes the state of the quanatum circuit system
    initQpe(circ, numQubits)

    # Creating and Applying a barrier for Qunatum Tunneling so the wavefunction of the quantum circuit can pass through a specified barrier setting limits
    # on the quantum circuit and splitting it up
    circ.barrier()

    # applying the inverse Fourier transform to find the original wavefunction based on all its parts
    reverse_qft(circ, numQubits)

    # Applying another barrier
    circ.barrier()

    # Measuring all the qubits except for the last one. This collapses the qubits in a state of superposition giving you a value based on probability in its wavefunction.
    circ.measure(range(numQubits), range(numQubits))

    # executes the quantum circuit 10000 times with our current simulator
    counts = run(circ, backend_=simulator, shots_=10000, optimization_level_=0)

    # Now since the values when measured and put through the circuit are based on probability, there are going to be a variety of values.
    # This counts the highest amount essentially giving you the most common answer of all the repetitions of the job
    max_counts_result = max(counts, key=counts.get)
    max_counts_result = int(max_counts_result, 2)

    # This is where we solve for the estimations of the actual phase and the result we are actually looking for.
    # So these calculations are based off of the fact that qubits have a superposition of 0 and 1, however using the hamarand gate
    # and setting the last qubit of one we know that the qubits must collapse into one. We also know that the measured values
    # depends on the phase and the number of qubits. Based on further properties of qubits and quantum mechanical systems,
    # We get these two formulas, "theta * 2^(number of qubits) = measured value" and "2*pi*theta = 1". We can now manipulate
    # these two formulas to get whatever value we want by substituting theta and changing values.
    theta = max_counts_result / (2 ** numQubits)
    if boolPi:
        return (1. / (2 * theta))
    else:
        periodDelta = input[1] - 2
        return (((1. / (pi * theta)) + periodDelta) ** (1 / userinput[0]))

#run the getUserinput function to modify the Userinput variable for use in the phase estimation
getUserinput()

#Makes a list for the range of the possibilities for the number of qubits used (2-12). It is impossible to have 1 because the a single qubit cannot be influenced by a Hadamard gate and = 1.
possibleQubit = list(range(2, maxqubits + 1))

#Debugging Tool
#print(possibleQubit)

#Makes a list for the values of calculation for printing and plotting
estimates = []
for q in possibleQubit:
    currentEstimate = phaseEstimate(q, userinput)
    estimates.append(currentEstimate)
#Debugging for for loop always printing 1 bug
#print(estimates)

#Making space for the results of the exact values
print("\n")
for p in possibleQubit:

    #Debugging Tools
    #print(estimates[p-1])
    #print(len(estimates))
    if boolPi:
        print("Using {} Qubits, the estimation for pi = {}".format(p, estimates[p-2]))
    else:
        print("Using {} Qubits, the estimation for {}√{} = {} ".format(p, userinput[0], userinput[1], estimates[p-2]) )


#Section of code devoted to Plotting the Estimations based on # of qubits
#If statement to determine if the user picked pi or not, then plotting the classical estimate based off the user's choice
if boolPi:
    plt.plot(possibleQubit, [pi] * len(possibleQubit), '--r')
else:
    plt.plot(possibleQubit, [userinput[1] ** (1 / userinput[0])] * len(possibleQubit), '--r')
plt.plot(possibleQubit, estimates, '.-', markersize=12)

#The x-limit is the same for the entire plot because the x-axis is just the # of qubits and that changes the same amount for pi and roots
plt.xlim([1.5, maxqubits+0.5])

#If statement to determine if the user picked pi or not, then plotting the y limit by giving it  a +1 range in both directions to the classic value
if boolPi:
    plt.ylim([pi - 1, pi + 1])
else:
    plt.ylim([(userinput[1] ** (1 / userinput[0])) - 1, (userinput[1] ** (1 / userinput[0])) + 1])

#If statement to determine if the user picked pi or not, then changing the legend to have pi LATEX notation or root LATEX notation
if boolPi:
    plt.legend(['Classical Estimate of $\pi$', 'Quantum Estimate of $\pi$'])
else:
    plt.legend(['Classical Estimate of $\sqrt[%d]{%d}$' % (userinput[0], userinput[1]), 'Quantum Estimate of $\sqrt[%d]{%d}$' % (userinput[0], userinput[1])])

#Labeling the x-axis number of qubits
plt.xlabel('Number of qubits', fontdict={'size': 20})

#Labeling the y-axis with LATEX notation depending on the user's choice.
if boolPi:
    plt.ylabel('Classical and Qunatum estimate of $\pi$', fontdict={'size': 20})
else:
    #formatting the string to have the proper user-inputted values while retaining LATEX notation
    plt.ylabel('Classical and Quantum estimate of $\sqrt[%d]{%d}$' % (userinput[0], userinput[1]))

#Setting basic axes
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)

#Showing the plot
plt.show()

